In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Load 'train.csv'**
train.csv 的資料為 12 個月中，每個月取 20 天，每天 24 小時的資料(每小時資料有 18 個 features)。

In [ ]:
# data = pd.read_csv('gdrive/My Drive/hw1-regression/train.csv', header = None, encoding = 'big5')
data = pd.read_csv('/kaggle/input/ml2020spring-hw1/train.csv', encoding = 'big5')

In [ ]:
# data.iloc[9]
# data.iloc[:17]

# **Preprocessing** 
取需要的數值部分，將 'RAINFALL' 欄位全部補 0。Rainfall属性的无降雨值为NR，降雨值为1. 将NR替换为数值0
另外，如果要在 colab 重覆這段程式碼的執行，請從頭開始執行(把上面的都重新跑一次)，以避免跑出不是自己要的結果（若自己寫程式不會遇到，但 colab 重複跑這段會一直往下取資料。意即第一次取原本資料的第三欄之後的資料，第二次取第一次取的資料掉三欄之後的資料，...）。

In [ ]:
# 去除前三列(日期/测站/测项)
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
#pandas to_numpy（）产生一个列表数组

In [ ]:
# print(raw_data)

# **Extract Features (1)**
属性组成18维向量
將原始 4320 * 18 的資料依照每個月分重組成 12 個月( 每个月20day*24h=480h横向小时 ), 18 (features) * 480 (hours) 的資料。
这样,每个月的资料在横向上是连续的时间

In [ ]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
        #取raw_data的行,对sample的列进行拼接
    month_data[month] = sample

In [ ]:
# print(month_data[0][0])

# **Extract Features (2)**
每个array是一个月, print(month_data[0][0])可查看第一个月第一天
每個月會有 480hrs，每 9 小時形成一個 data，每個月會有 471 個 data，故總資料數為 471 * 12 筆，而每筆 data 有 9 * 18 的 features (一小時 18 個 features * 9 小時)。

對應的 target 則有 471 * 12 個(第 10 個小時的 PM2.5)

前9h数据作为train_x, 第10h的pm2.5作为train_y, 制作xy样本

In [ ]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:#最后一天的最后10h是最后一组样本了,再往后无法作样本了
                continue
            #-1在这里应该可以理解为一个正整数通配符，它代替任何整数. reshape(1, -1)化为一行
            # print(month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1))
            #x分片时是所有行(属性), 如果不进行reshape, 则每个属性的9小时单独成1行
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value第10个属性是PM2.5
print(x)
print(y)

# **Normalize (1)**
预处理数据有一个很关键的步骤就是数据的标准化
许多学习算法中目标函数的基础都是假设所有的特征都是零均值并且具有同一阶数上的方差。如果某个特征的方差比其他特征大几个数量级，那么它就会在学习算法中占据主导位置，导致学习器并不能像我们说期望的那样，从其他特征中学习。
## 1 StandardScaler
标准化数据通过减去均值然后除以方差（或标准差），这种数据标准化方法经过处理后数据符合标准正态分布，即均值为0，标准差为1
## 2 MinMaxScaler
这种方法是对原始数据的线性变换，将数据归一到[0,1]中间
## 3 RobustScaler
如果你的数据包含许多异常值，使用均值和方差缩放可能并不是一个很好的选择。
This Scaler removes the median（中位数） and scales the data according to the quantile range(四分位距离，也就是说排除了outliers)

In [ ]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 标准差
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

# **Split Training Data Into "train_set" and "validation_set"**
這部分是針對作業中第二題、第三題，生成比較中用來訓練的 train_set 和不會被放入訓練、只是用來驗證的 validation_set。

In [ ]:
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]#取80%的样本, floor向下取整. 0~0.8
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]#0.8~1
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

In [ ]:
# # 题目4-数据划分-只取后5h-cell
# #只能运行一次
# x_5h = x[:, (4*18):]
# #test测试集数据预处理
# testdata = pd.read_csv('/kaggle/input/ml2020spring-hw1/test.csv', header = None, encoding = 'big5')
# test_data = testdata.iloc[:, 2:]#只能运行一次
# test_data[test_data == 'NR'] = 0
# test_data = test_data.to_numpy()

In [ ]:
# # 题目4-测试数据形状改变-只取后5h-cell
# test_x = np.empty([240, 18*9], dtype = float)
# # test_x_5h = np.empty([240, 18*5], dtype = float)
# # 10 小時為一筆，前九小時作x的feature. 一共取出 240 筆不重複的 test data，
# for i in range(240):#每一笔分别存, 一笔中有18个feature
#     test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)#原数据的小时在列中体现, 把所有前9小时都存入了
# #     test_x_5h[i, :] = test_data[18 * i: 18* (i + 1), 4:].reshape(1, -1)#只取后5h
# test_x_5h = test_x[:, (4*18):]

In [ ]:
# # 题目4-模型改进-只取后5h-cell
# dim_5h = 18 * 5 + 1
# w_5h = np.zeros([dim_5h, 1])
# n_train_set = math.floor(12 * 471)
# #train
# learning_rate = 2
# iter_time = 1000
# eps = 0.0000000001
# #np.ones全部赋为1, concatenate数组拼接
# x_5h = np.concatenate((np.ones([n_train_set, 1]), x_5h), axis = 1).astype(float)
# #train
# adagrad_5h = np.zeros([dim_5h, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss = np.sqrt(np.sum(np.power(np.dot(x_5h, w_5h) - y, 2))/n_train_set)#rmse标准差
#     #每进行100次迭代, 输出loss值
#     if(t%100==0):
#         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient_5h = 2 * np.dot(x_5h.transpose(), np.dot(x_5h, w_5h) - y) #dim*1
#     adagrad_5h += gradient_5h ** 2
#     w_5h = w_5h - learning_rate * gradient_5h / np.sqrt(adagrad_5h + eps)
# np.save('weight_5h.npy', w_5h)
# w_5h

In [ ]:
# # 题目4-测试集预测-只取后5h-cell
# #测试集输入数据标准化
# mean_x_5h = np.mean(test_x_5h, axis = 0)
# std_x_5h = np.std(test_x_5h, axis = 0) #标准差
# for i in range(len(test_x_5h)):
#     for j in range(len(test_x_5h[0])):
#         if std_x_5h[j] != 0:
#             test_x_5h[i][j] = (test_x_5h[i][j] - mean_x_5h[j]) / std_x_5h[j]
# test_x_5h = np.concatenate((np.ones([240, 1]), test_x_5h), axis = 1).astype(float)

In [ ]:
# # 题目4-测试集预测-只取后5h-cell
# #预测
# w_5h = np.load('weight_5h.npy')
# ans_y_5h = np.dot(test_x_5h, w_5h)
# # ans_y_5h
# #保存预测数据
# import csv
# with open('submit.csv', mode='w', newline='') as submit_file:
#     csv_writer = csv.writer(submit_file)
#     header = ['id', 'value']
#     print(header)
#     csv_writer.writerow(header)
#     for i in range(240):
#         row = ['id_' + str(i), ans_y_5h[i][0]]
#         csv_writer.writerow(row)
#         print(row)

In [ ]:
# # 题目3-PM2.5输入特征划分-cell
# n_train_set = math.floor(12 * 471 * 0.8)
# n_validation_set = math.ceil(12 * 471 * 0.2)

# # x = np.empty([12 * 471, 18 * 9], dtype = float)
# x_train_set_PM = np.empty([n_train_set, 1 * 9], dtype = float)
# x_validation_PM = np.empty([n_validation_set, 1 * 9], dtype = float)
# for hour in range(9):#每笔输入数据中的小时9, 和提取PM2.5所在的行数9, 在理解的时候不要搞混
#     x_train_set_PM[:,hour] = x_train_set[:, 18*hour + 9]
#     x_validation_PM[:,hour] = x_validation[:, 18*hour + 9]
# x_train_set_PM, x_validation_PM

In [ ]:
# # 题目3-全属性对比只输入PM2.5数据-cell
# # 训练
# dim = 18 * 9 + 1
# w = np.zeros([dim, 1])
# x_train_set = np.concatenate((np.ones([n_train_set, 1]), x_train_set), axis = 1).astype(float)
# # x_train_set
# #train
# learning_rate = 100
# iter_time = 1000
# adagrad = np.zeros([dim, 1])
# eps = 0.0000000001
# for t in range(iter_time):
#     loss = np.sqrt(np.sum(np.power(np.dot(x_train_set, w) - y_train_set, 2))/n_train_set)#rmse标准差也要根据划分后的数据笔数修改所除的n
#     #每进行100次迭代, 输出loss值
# #     if(t%100==0):
# #         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x_train_set.transpose(), np.dot(x_train_set, w) - y_train_set) #dim*1
#     adagrad += gradient ** 2
#     w = w - learning_rate * gradient / np.sqrt(adagrad + eps)

In [ ]:
# # 题目3-cell
# # PM2.5训练
# dim_PM = 1 * 9 + 1
# w_PM = np.zeros([dim_PM, 1])
# #np.ones全部赋为1, concatenate数组拼接
# x_train_set_PM = np.concatenate((np.ones([n_train_set, 1]), x_train_set_PM), axis = 1).astype(float)
# #train
# adagrad_PM = np.zeros([dim_PM, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss = np.sqrt(np.sum(np.power(np.dot(x_train_set_PM, w_PM) - y_train_set, 2))/n_train_set)#rmse标准差
#     #每进行100次迭代, 输出loss值
# #     if(t%100==0):
# #         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient_PM = 2 * np.dot(x_train_set_PM.transpose(), np.dot(x_train_set_PM, w_PM) - y_train_set) #dim*1
#     adagrad_PM += gradient_PM ** 2
#     w_PM = w_PM - learning_rate * gradient_PM / np.sqrt(adagrad_PM + eps)
    
    
# # 测试
# x_validation = np.concatenate((np.ones([n_validation_set, 1]), x_validation), axis = 1).astype(float)
# ans_y = np.dot(x_validation, w)
# # PM2.5
# x_validation_PM = np.concatenate((np.ones([n_validation_set, 1]), x_validation_PM), axis = 1).astype(float)
# ans_y_PM = np.dot(x_validation_PM, w_PM)


# # 分别计算平均误差
# n_vali=len(ans_y)
# average_error=0
# for i in range(0, n_vali):  
#     average_error+=abs(y_validation[i][0]-ans_y[i][0])
# average_error/=n_vali
# # PM2.5
# average_error_PM=0
# for i in range(0, n_vali):  
#     average_error_PM+=abs(y_validation[i][0]-ans_y_PM[i][0])
# average_error_PM/=n_vali

# print(average_error)
# print(average_error_PM)

In [ ]:
# # 题目2-5小时输入划分-cell
# x_train_set_5h = x_train_set[:, (4*18):]
# x_validation_5h = x_validation[:, (4*18):]

In [ ]:
# #题目2-5h,9h对比数据训练-cell
# dim = 18 * 9 + 1
# n_train_set = math.floor(12 * 471 * 0.8)
# w = np.zeros([dim, 1])
# x_train_set = np.concatenate((np.ones([n_train_set, 1]), x_train_set), axis = 1).astype(float)
# # x_train_set
# #train
# learning_rate = 100
# iter_time = 1000
# adagrad = np.zeros([dim, 1])
# eps = 0.0000000001
# for t in range(iter_time):
#     loss = np.sqrt(np.sum(np.power(np.dot(x_train_set, w) - y_train_set, 2))/n_train_set)#rmse标准差也要根据划分后的数据笔数修改所除的n
#     #每进行100次迭代, 输出loss值
#     if(t%100==0):
#         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x_train_set.transpose(), np.dot(x_train_set, w) - y_train_set) #dim*1
#     adagrad += gradient ** 2
#     w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
# # 5h
# dim_5h = 18 * 5 + 1
# w_5h = np.zeros([dim_5h, 1])
# #np.ones全部赋为1, concatenate数组拼接
# x_train_set_5h = np.concatenate((np.ones([n_train_set, 1]), x_train_set_5h), axis = 1).astype(float)
# #train
# adagrad_5h = np.zeros([dim_5h, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss = np.sqrt(np.sum(np.power(np.dot(x_train_set_5h, w_5h) - y_train_set, 2))/n_train_set)#rmse标准差
#     #每进行100次迭代, 输出loss值
# #     if(t%100==0):
# #         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient_5h = 2 * np.dot(x_train_set_5h.transpose(), np.dot(x_train_set_5h, w_5h) - y_train_set) #dim*1
#     adagrad_5h += gradient_5h ** 2
#     w_5h = w_5h - learning_rate * gradient_5h / np.sqrt(adagrad_5h + eps)

In [ ]:
# #题目2-5h,9h对比数据测试-cell
# #test
# x_validation = np.concatenate((np.ones([math.ceil(12 * 471 * 0.2), 1]), x_validation), axis = 1).astype(float)
# # test_x
# # w = np.load('weight-by-train-set.npy')
# ans_y = np.dot(x_validation, w)
# # ans_y
# # 5h
# x_validation_5h = np.concatenate((np.ones([math.ceil(12 * 471 * 0.2), 1]), x_validation_5h), axis = 1).astype(float)
# ans_y_5h = np.dot(x_validation_5h, w_5h)
# ans_y_5h

In [ ]:
# #题目2-分别计算平均误差-cell
# n_vali=len(ans_y)
# average_error=0
# for i in range(0, n_vali):  
#     average_error+=abs(y_validation[i][0]-ans_y[i][0])
# average_error/=n_vali
# # 5h
# average_error_5h=0
# for i in range(0, n_vali):  
#     average_error_5h+=abs(y_validation[i][0]-ans_y_5h[i][0])
# average_error_5h/=n_vali

# print(average_error)
# print(average_error_5h)

# **Training**

下面的 code 採用 Root Mean Square Error

因為常數項的存在，所以 dimension (dim) 需要多加一欄；eps 項是避免 adagrad 的分母為 0 而加的極小數值。

每一個 dimension (dim) 會對應到各自的 gradient, weight (w)，透過一次次的 iteration (iter_time) 學習。

In [ ]:
# baseline-cell
dim = 18 * 9 + 1
w = np.zeros([dim, 1])
#np.ones全部赋为1, concatenate数组拼接
x = np.concatenate((np.ones([12 * 471, 1]), x), axis = 1).astype(float)
x

In [ ]:
# #题目1-作图cell
# learning_rate = [0.5, 2, 50, 100]
# iter_time = 1000
# adagrad = np.zeros([dim, 1])
# eps = 0.0000000001
# model_data = {
#     't': [ ],
#     'loss1': [ ],
#     'loss2': [ ],
#     'loss3': [ ],
#     'loss4': [ ]
# }
# w1=w#因为作图时不同学习率都要用到w,adagrad,需要留底或重置
# w2=w
# w3=w
# w4=w
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss1 = np.sqrt(np.sum(np.power(np.dot(x, w1) - y, 2))/471/12)#rmse标准差
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x.transpose(), np.dot(x, w1) - y) #dim*1
#     adagrad += gradient ** 2
#     w1 = w1 - learning_rate[0] * gradient / np.sqrt(adagrad + eps)
#     model_data['t'].append(t)
#     model_data['loss1'].append(loss1)
# adagrad = np.zeros([dim, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss2 = np.sqrt(np.sum(np.power(np.dot(x, w2) - y, 2))/471/12)#rmse标准差
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x.transpose(), np.dot(x, w2) - y) #dim*1
#     adagrad += gradient ** 2
#     w2 = w2 - learning_rate[1] * gradient / np.sqrt(adagrad + eps)
#     model_data['loss2'].append(loss2)
# adagrad = np.zeros([dim, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss3 = np.sqrt(np.sum(np.power(np.dot(x, w3) - y, 2))/471/12)#rmse标准差
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x.transpose(), np.dot(x, w3) - y) #dim*1
#     adagrad += gradient ** 2
#     w3 = w3 - learning_rate[2] * gradient / np.sqrt(adagrad + eps)
#     model_data['loss3'].append(loss3)
# adagrad = np.zeros([dim, 1])
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss4 = np.sqrt(np.sum(np.power(np.dot(x, w4) - y, 2))/471/12)#rmse标准差
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x.transpose(), np.dot(x, w4) - y) #dim*1
#     adagrad += gradient ** 2
#     w4 = w4 - learning_rate[3] * gradient / np.sqrt(adagrad + eps)
#     model_data['loss4'].append(loss4)
# # model_data

In [ ]:
# #题目1-作图cell
# import matplotlib.pyplot as plt
 
# #读取数据
# # model_data = {
# #     't': [1,2,3],
# #     'loss': [7.1,23.2,11.8]
# # }
# plt.figure(figsize=(10,5))#设置画布的尺寸
# plt.title('Loss-Iteration Line Chart',fontsize=20)#标题，并设定字号大小
# plt.xlabel(u'x-iteration times',fontsize=14)#设置x轴，并设定字号大小
# plt.ylabel(u'y-loss',fontsize=14)#设置y轴，并设定字号大小
# plt.xlim((0, 1000))
# plt.ylim((5, 15))
# #color：颜色，linewidth：线宽，linestyle：线条类型，label：图例，marker：数据点的类型
# plt.plot(model_data['t'],model_data['loss1'],color="darkblue",linewidth=1,linestyle='--',label='learning_rate=0.5', marker='+')
# plt.plot(model_data['t'],model_data['loss2'],color="deeppink",linewidth=2,linestyle=':',label='learning_rate=2', marker='o')
# plt.plot(model_data['t'],model_data['loss3'],color="goldenrod",linewidth=1.5,linestyle='-',label='learning_rate=50', marker='*')
# plt.plot(model_data['t'],model_data['loss4'],color="green",linewidth=1.5,linestyle='-',label='learning_rate=100', marker='*')

 
# plt.legend(loc=1)#图例展示位置，数字代表第几象限
# plt.show()#显示图像

In [ ]:
# # baseline-cell
# learning_rate = 100
# iter_time = 1000
# adagrad = np.zeros([dim, 1])
# eps = 0.0000000001
# for t in range(iter_time):
#     #np.dot矩阵点积, power:x^y
#     loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse标准差
#     #每进行100次迭代, 输出loss值
#     if(t%100==0):
#         print(str(t) + ":" + str(loss))
#     #transpose矩阵转置
#     gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
#     adagrad += gradient ** 2
#     w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
# np.save('weight.npy', w)
# w

In [ ]:
# beat-baseline-cell
learning_rate = 1
iter_time = 1200
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    #np.dot矩阵点积, power:x^y
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse标准差
    #每进行100次迭代, 输出loss值
    if(t%100==0):
        print(str(t) + ":" + str(loss))
    #transpose矩阵转置
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w

# **Testing**
載入 test data，並且以相似於訓練資料預先處理和特徵萃取的方式處理，使 test data 形成 240 個維度為 18 * 9 + 1 的資料。

读取csv文件,输入x每18行加一个常数项维度, 加上前9小时数据

输出矩阵weight vector与x的点乘

In [ ]:
# baseline-cell
# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv('/kaggle/input/ml2020spring-hw1/test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()

In [ ]:
# baseline-cell
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)

In [ ]:
# baseline-cell
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

In [ ]:
# # beat-baseline-cell
# mean_test_x = np.mean(test_x, axis = 0) #18 * 9 
# std_test_x = np.std(test_x, axis = 0) #18 * 9 标准差
# for i in range(len(test_x)):
#     for j in range(len(test_x[0])):
#         if std_test_x[j] != 0:
#             test_x[i][j] = (test_x[i][j] - mean_test_x[j]) / std_test_x[j]
# test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
# test_x

# **Prediction**
weight 和測試資料預測 target。

In [ ]:
# baseline-cell
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

# **Save Prediction to CSV File**

In [ ]:
# baseline-cell
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)